# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [3]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('D:/ML/Dataset/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('D:/ML/Dataset/emails/ham', 'ham'))


Let's have a look at that DataFrame:

In [4]:
data.head()

,class,message
D:/ML/Dataset/emails/spam\00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
D:/ML/Dataset/emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
D:/ML/Dataset/emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
D:/ML/Dataset/emails/spam\00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
D:/ML/Dataset/emails/spam\00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['message'], data['class'], test_size=0.33, random_state=42)
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(X_train.values)

classifier = MultinomialNB()
targets = y_train.values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [18]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'],
      dtype='<U4')

In [19]:

l=[]
for i in X_test:
    l.append(i)
l_counts = vectorizer.transform(l)
predicts = classifier.predict(l_counts)
correct=0
wrong=0
for i,j in zip(predicts,y_test):
    if(i==j):
        correct=correct+1
    else:
        wrong=wrong+1
print(correct,wrong)

945 45


In [27]:
#ERROR RATE:
accuracy = (abs(correct-len(l))/len(l))*100
print("Error Rate:",str(accuracy),"%","\nACCURACY:",str(100-accuracy),"%")

Error Rate: 4.545454545454546 % 
ACCURACY: 95.45454545454545 %


## Activity